In [1]:
import os
import re
import json
import langchain

In [20]:
from langchain_community.document_loaders import JSONLoader
loader = JSONLoader(
    file_path='temp.json',
    jq_schema='.[]',
    text_content=False
)

docs = loader.load()


for index,doc in enumerate(docs):
    temp = json.loads(doc.page_content)
    data = ""
    del doc.metadata['source']
    del doc.metadata['seq_num']    

    for key, value in temp.items():
        if key != "description" and key != "course_link":
            if isinstance(value, list):
                # Join string representations of list items with spaces
                value = ' '.join(str(v).lower() for v in value)
            elif isinstance(value, str):
                value = value.lower()
            doc.metadata[key] = value
    
    for key, value in temp.items():
        if key == "description":
            if value is not None:
                # Replace newlines with spaces for uniformity
                desc = value.replace('\n', ' ')
                # Remove unnecessary '*' and '-' characters
                desc = re.sub(r'[\*\-]', '', desc)
                # Remove extra spaces
                desc = re.sub(r'\s+', ' ', desc).strip()
                # Split on bullet points (•)
                bullet_parts = [part.strip() for part in re.split(r'•', desc) if part.strip()]
                sentences = []
                for part in bullet_parts:
                    # Instead of splitting on every period, only split on periods that are not part of ordered lists (e.g., "a.", "1.")
                    # We'll use a regex to split on periods that are NOT preceded by a single letter/number and a space
                    # This will keep "a. " or "1. " together
                    sub_sentences = re.split(r'(?<!\b[a-zA-Z0-9])\.(?!\d)', part)
                    for s in sub_sentences:
                        s = s.strip()
                        if s:
                            # Ensure each sentence ends with a period
                            if not s.endswith('.'):
                                s += '.'
                            # Convert to lowercase and remove non-alphabetic characters (except spaces)
                            s = s.lower()
                            s = re.sub(r'[^a-z\s]', '', s)
                            # Remove extra spaces again after removing non-alphabetic chars
                            s = re.sub(r'\s+', ' ', s).strip()
                            if s:  # Only add non-empty sentences
                                sentences.append(s)
                # Reconstruct with each sentence on a new line
                updated_description = '\n'.join(sentences)
                data += f"{key} : {updated_description}"
            else:
                data += f"{key} : "
    del doc.metadata["class_time"]
    doc.metadata.update({"index":f"{index}"})
    doc.page_content = data


In [21]:
for doc in docs:
    print(doc.metadata)

{'course_code': 'ae102', 'course_name': 'data analysis and interpretation', 'department': 'aerospace engineering', 'instructors': '', 'tags': '', 'credits': 6, 'prerequisites': '', 'is_running': False, 'venue': '', 'duration': '', 'slot': '', 'index': '0'}
{'course_code': 'ae103', 'course_name': 'a historical perspective of aerospace engineering', 'department': 'aerospace engineering', 'instructors': 'chandra sekher yerramalli', 'tags': 'theory', 'credits': 6, 'prerequisites': '', 'is_running': True, 'venue': 'ic 1', 'duration': 'fullsemester', 'slot': '4', 'index': '1'}
{'course_code': 'ae152', 'course_name': 'introduction to aerospace engg', 'department': 'aerospace engineering', 'instructors': 'rajkumar sureshchandra pant', 'tags': 'theory', 'credits': 6, 'prerequisites': '', 'is_running': True, 'venue': 'la 001', 'duration': 'fullsemester', 'slot': '5', 'index': '2'}
{'course_code': 'ae153', 'course_name': 'introduction to aerospace engg', 'department': 'aerospace engineering', 'in

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)

c:\Users\happy\Downloads\Course-Buddy-AI\course_buddy_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
vector_store.delete_collection()

In [3]:
from langchain.vectorstores import Chroma

vector_store = Chroma(
    embedding_function = embedding_model,
    persist_directory='course_vector_database',
    collection_name = 'sample'
)

C:\Users\happy\AppData\Local\Temp\ipykernel_13440\868050241.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [28]:
vector_store.get(include=['metadatas'])

{'ids': [],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'included': ['metadatas'],
 'data': None,
 'metadatas': []}

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=15)
chunks = text_splitter.split_documents(docs)

In [30]:
len(chunks)

600

In [31]:
vector_store.add_documents(chunks)

['e47ffd6e-710d-446c-b993-fb91c8c9d75d',
 '4bcf173b-4c8e-49f7-9be9-efe79cbbbef9',
 '8904ed94-452d-4069-b602-45bab42a9cb3',
 'd10ee16d-0803-49af-98c1-2f02b6f0de5f',
 'ed958695-fd2e-43f5-b4fd-4d756f0b9edf',
 'dfe049f1-d8b9-4bec-be13-01d4e4a062f0',
 '1b677493-4a98-4794-a265-e32e18a9d5e2',
 '6f7bc3e8-5680-480d-8f52-660f784cfda7',
 'e74414b3-e01c-4c2b-bcf0-fb63a705caeb',
 'c080d440-2f1b-4fbf-b70f-c03ec60324b1',
 '9f92dbc6-626f-4d97-8af3-35e7b36a38e3',
 '5057ff9f-98c0-4cde-98fc-c0d18d1e14c5',
 '2d8ec04a-ff20-4cc1-bbc1-a6b6964129fe',
 'c7567247-c6bd-42f9-86aa-e65ef37cbc66',
 '77007a2b-dba0-4a75-8f96-09a415cff3ab',
 'f15fdd3a-780e-4bdc-b9a0-6129b112367c',
 '3680e9e8-3e18-487f-87a3-bdf0da637e72',
 'fba80c6b-5da0-4ee7-a1f7-09f736afa695',
 '1ea442f8-b24e-4737-9ddb-cd4886cd98a6',
 '948033fe-9c99-4d28-adf2-871071fc1802',
 '7b51354e-15fe-4aba-8d4a-434b08f9a9cf',
 'ac5b025e-e39b-443a-9673-3fa26e99dd5a',
 '38b66a2f-9d0b-4ff7-a298-0fa1bc063634',
 'a5d1a00d-4cbb-4f02-a72c-14d6b7efae3c',
 'c1ea1541-707d-

In [4]:
import os
from dotenv import load_dotenv

# This line loads the variables from your .env file into the environment
load_dotenv()

# You can now access your key using os.getenv()
my_api_key = os.getenv("GOOGLE_API_KEY")

# You can verify it's loaded (optional)
if my_api_key:
    print("✅ API Key loaded successfully!")
    # Your LangChain or other code that needs the key can now run.
    # It will often find the key automatically from the environment.
else:
    print("❌ Could not load API Key. Check your .env file and path.")

# Example: Using it with LangChain
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # This automatically uses the loaded key

✅ API Key loaded successfully!


In [5]:
dept_lingos = {
    'aerospace engineering': [
        'aero', 'aerospace', 'ae', 'aero engg', 'aero eng', 'aero dept', 'aerospace dept',
        'aero department', 'aerospace engineering', 'aeronautical', 'aeronautics'
    ],
    'chemical engineering': [
        'chem engg', 'chem eng', 'che', 'chem', 'chemical', 'chem dept', 'chemical dept',
        'chemical engineering', 'chemical eng', 'chemistry engg', 'che engg'
    ],
    'climate studies': [
        'climate', 'climate studies', 'climate sci', 'climate dept', 'climate department',
        'csu', 'climate engg'
    ],
    'computer science and engineering': [
        'cse', 'cs', 'comp sci', 'computer science', 'cs engg', 'cs eng', 'cs dept',
        'comp science', 'comp engg', 'computers dept', 'cse dept', 'comps'
    ],
    'industrial design': [
        'idc', 'ind design', 'industrial design', 'design dept', 'design department'
    ],
    'centre for digital health': [
        'cdh', 'digital health', 'health tech', 'digital healthcare', 'dig health'
    ],
    'centre for machine intelligence and data science': [
        'cminds', 'machine intelligence', 'ai dept', 'data science', 'ml dept',
        'c-minds', 'machine learning', 'artificial intelligence', 'datasci'
    ],
    'economics': [
        'eco', 'economics', 'eco dept', 'economics dept', 'economics department'
    ],
    'electrical engineering': [
        'ee', 'electrical', 'elec', 'elec engg', 'elec eng', 'ee dept', 'electrical dept',
        'electrical engineering', 'electrical eng', 'elektical', 'elec department'
    ],
    'energy science and engineering': [
        'ese', 'energy engg', 'energy sci', 'energy science', 'energy dept',
        'energy sciences', 'energy engineering'
    ],
    'bioscience and bioengineering': [
        'bio', 'bio engg', 'bsbe', 'bio sciences', 'bio sci', 'bio dept', 'bioengineering',
        'biotech', 'biological sciences'
    ],
    'centre for entrepreneurship': [
        'cfep', 'entrepreneurship', 'entrepreneurship centre', 'startup centre',
        'e-cell', 'entrepreneur dept'
    ],
    'engineering physics': [
        'ep', 'engg physics', 'phy engg', 'eng phy', 'engineering physics',
        'physics engg'
    ],
    'environmental science and engineering': [
        'env sci', 'ese', 'environmental engg', 'environmental science', 'env dept',
        'environment engg', 'environment sci'
    ],
    'chemistry': [
        'chem', 'chem dept', 'chemistry', 'chemistry dept', 'chemistry department'
    ],
    'educational technology': [
        'ed tech', 'educational tech', 'edutech', 'edu tech', 'education technology'
    ],
    'centre of studies in resources engineering': [
        'csre', 'resources engg', 'resources engineering', 'resources dept'
    ],
    'applied geophysics': [
        'geo', 'applied geo', 'geophysics', 'geo phys', 'geo dept', 'applied geophysics'
    ],
    'earth sciences': [
        'earth sci', 'es', 'geology', 'earth dept', 'earth science', 'geological sciences'
    ],
    'humanities and social sciences': [
        'hss', 'hum', 'humanities', 'social sci', 'humanities dept', 'humanities department',
        'social sciences', 'arts dept'
    ],
    'industrial engineering and operations research': [
        'ieor', 'operations research', 'ind engg', 'ind eng', 'or', 'operations dept',
        'industrial engineering'
    ],
    'shailesh j. mehta school of management': [
        'sjmsom', 'som', 'management school', 'mba dept', 'school of management',
        'business school', 'b-school'
    ],
    'centre for liberal education (cledu)': [
        'cledu', 'liberal education', 'liberal studies', 'liberal dept'
    ],
    'mathematics': [
        'math', 'maths', 'math dept', 'math department', 'mathematics'
    ],
    'mechanical engineering': [
        'mech', 'mech engg', 'me', 'mechanical', 'mechanical dept', 'mechanical department',
        'mechnical', 'mech engineering', 'berozgar'
    ],
    'metallurgical engineering and materials science': [
        'meta', 'metallurgy', 'mems', 'materials sci', 'materials science',
        'metallurgy dept', 'metallurgical engineering'
    ],
    'centre for research in nanotechnology and science': [
        'crnts', 'nano tech', 'nanotech', 'nanotechnology', 'nano dept'
    ],
    'physics': [
        'phy', 'physics', 'physics dept', 'physics department'
    ],
    'centre for policy studies': [
        'cps', 'policy studies', 'policy centre', 'policy dept'
    ],
    'systems and control engineering': [
        'syscon', 'sce', 'systems engg', 'control engg', 'systems and control',
        'control systems'
    ],
    'applied statistics and informatics': [
        'asi', 'stats', 'statistics', 'stats dept', 'statistics dept', 'applied stats'
    ],
    'centre for technology alternatives for rural areas': [
        'ctara', 'rural tech', 'rural development centre', 'rural development'
    ],
    'centre for urban science and engineering': [
        'cuse', 'urban science', 'urban studies', 'urban engg', 'urban dept'
    ],
    'visual communication': [
        'viscom', 'visual comm', 'visual design', 'visual communication'
    ],
    'civil engineering': [
        'civil', 'ce', 'civil engg', 'civil dept', 'civil engineering', 'mistri', 'mistri department'
    ]
}


In [6]:
def handle_department_lingo(search_filter, query):
    if "department" in search_filter:
        dept_val = search_filter["department"].strip().lower()
    
        found_lingo = False
        for key, lingos in dept_lingos.items():
            if dept_val in lingos:
                # print(f"Found lingo: '{dept_val}' maps to '{key}'")
                # Update the filter to use the official department name
                search_filter["department"] = key
                # Append the clarification to the query for the retriever
                query += f" (Note: '{dept_val}' refers to the {key} department)"
                found_lingo = True
                break
        
        if not found_lingo:
            print("No lingo found, using department value as is.")
    return search_filter, query

In [7]:
def courses_by_topics(search_filter, descriptive_query):
    if len(search_filter) == 1:
        filter_arg = search_filter
    else:
        filter_arg = {'$and': [{k:v} for k,v in search_filter.items()]}

    retriever = vector_store.as_retriever(search_kwargs={"filter": filter_arg, "k": 100})
    filtered_docs = retriever.get_relevant_documents("")

    filtered_ids = list(set([doc.metadata.get("index") for doc in filtered_docs]))

    filtered_ids = [id_ for id_ in filtered_ids if id_ is not None]

    if filtered_ids:
        id_filter = {"index": {"$in": filtered_ids}}
        print(id_filter)
        # Now do semantic search within these filtered docs
        semantic_retriever = vector_store.as_retriever(
            search_kwargs={"filter": id_filter, "k": 5}
        )
        # Use the descriptive_query for semantic search
        retrieved_docs = semantic_retriever.get_relevant_documents(descriptive_query)
    else:
        print("No valid IDs found in filtered docs for semantic search.")
        retrieved_docs = []
    return retrieved_docs

In [8]:
from rapidfuzz import fuzz

def instructor_fuzzy_match(doc_instructors, search_instructors):
    # Both are lists of instructor names (strings)
    for search_name in search_instructors:
        for doc_name in doc_instructors:
            if fuzz.token_sort_ratio(doc_name.lower(), search_name.lower()) >= 80:
                return True
    return False

In [32]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# --- 1. SETUP ---
load_dotenv()

gemini_router_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
gemini_generator_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7) 


history_aware_prompt = ChatPromptTemplate.from_messages([
    ("system", """Your task is to rephrase a follow-up user question into a self-contained, standalone question based on the provided chat history.

    **CRITICAL RULE: Do NOT answer the user's question. Your ONLY job is to reformulate the question itself.**

    Here is an example:

    <Chat History>
    Human: "what is the course content of CS 772"
    AI: "CS 772 covers topics like dynamic programming, graph algorithms, and NP-completeness."
    </Chat History>

    <User Question>
    "Any course similar to that one?"
    </User Question>

    <Standalone Question>
    "Are there any courses similar to CS 772?"
    </Standalone Question>

    If the user's question is already standalone, return it unchanged.
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

rewrite_query_chain = history_aware_prompt | gemini_router_llm | StrOutputParser()

# --- 3. PRE-RETRIEVAL ROUTER SETUP ---
class CourseMetadataSearch(BaseModel):
    """A tool to find courses using filters like instructor, course code, slot, or department."""
    course_code: str = Field(default=None, description="The code for the course, e.g., 'bb706'")
    instructors: str = Field(default=None, description="The name of the instructor")
    slot: str = Field(default=None, description="The schedule slot for the course, e.g., '6'")
    department: str = Field(default=None, description="The department offering the course, e.g., 'Physics', 'Mathematics', or 'Bioscience and Engineering'")
    user_intent: str = Field(default=None, description='Tells us if the user wants courses similar to some course, a general enquiry of some course, or is just chatting. Should be one of "get_course_detail", "similar_courses", "courses_by_topics", or "chatting".')
    descriptive_query: str = Field(default=None, description = "The enhanced user query for better semantic search, containing only the topics to be taught in the course.")

    def __init__(self, **data):
        super().__init__(**data)
        # Standardize fields to lowercase for consistent filtering
        if self.course_code is not None:
            self.course_code = self.course_code.lower().replace(" ", "").replace("-", "")
        if self.instructors is not None:
            self.instructors = self.instructors.lower()
        if self.slot is not None:
            self.slot = self.slot.lower()
        if self.department is not None:
            self.department = self.department.lower()

llm_with_tool = gemini_router_llm.with_structured_output(CourseMetadataSearch)

router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Analyze the user's query to determine whether they are asking for specific course details based on course code, professor, or slot. 
    If a specific course is identified, extract the relevant details. 
    Identify and extract:
    - Instructor name (e.g. prof. ramesh chandra -> ramesh chandra)
    - Course code (e.g. BB-706, bb 706, Bb 706 or bb   706 to bb706 i.e. to lowercase and no gap between characters and numbers)
    - Department (If the query contains a department mention (full name, slang, or abbreviation), return it **exactly as written** by the user just remove the "department", "dept" or similar keywords from it. )
    - Slot number 
    - user intent (Tells us if the user is seeking similar courses, wants to query about some course, or wants to know about the chatbot -> "get_course_detail", "similar_courses", "courses_by_topics" or "chatting")
    - descriptive query (contains only the topics to be taught in course.)
    """),
    ("human", "{query}")
])
pre_retrieval_router_chain = router_prompt | llm_with_tool

# --- 4. FINAL ANSWER GENERATION CHAIN ---
final_prompt_template = """
You are a helpful college course assistant named Course Buddy AI, assisting students with course-related questions and helping them find courses of interest. Answer the user's question based ONLY on the following context and chat history.

If the context is empty:
- First, check the chat history (HISTORY below). If you find any previous conversation or information in the chat history that is relevant to the user's current question, use it to answer appropriately.
- If nothing relevant is found in the chat history:
    - If the user's query is about a course (e.g., asking for course details, information, or recommendations), apologize and state that you couldn't find information on that specific topic.
    - If the user's query is general chatting (not about a course), respond appropriately as a friendly assistant (e.g., introduce yourself, offer help, or answer the chatty question).

HISTORY:
{chat_history}

CONTEXT:
{context}

QUESTION:
{question}

YOUR ANSWER:
"""
chat_history = []

final_prompt = ChatPromptTemplate.from_template(final_prompt_template)
final_rag_chain = final_prompt | gemini_generator_llm | StrOutputParser()

In [35]:
# --- 5. THE COMPLETE APPLICATION LOGIC ---
from langchain_core.runnables import history

# NOTE: The chat_history variable is initialized as a list at the top of the notebook.
# It is used to store the conversation history between the user and the assistant.
# Each user message is appended as a HumanMessage, and each assistant response as an AIMessage.

def ask_course_bot(query: str):
    """
    This function orchestrates the entire RAG pipeline.
    """
    print(f"\n🤔 Query: '{query}'")

    query = rewrite_query_chain.invoke({
        "chat_history": chat_history,
        "question": query
    })
    print(f"Rewritten query: '{query}'")

    # Step 1: Call the pre-retrieval router
    router_decision = pre_retrieval_router_chain.invoke({"query": query})
    search_filter = {key: value for key, value in router_decision.dict().items() if value is not None}
    
    print(search_filter,'\n')
    search_filter, query = handle_department_lingo(search_filter, query)

    print(f"⚙️ Search Filter: {search_filter}")

    # Defensive: Check for 'user_intent' key existence
    user_intent = search_filter.get('user_intent', None)
    if not user_intent:
        print("❌ Error: 'user_intent' not found in search_filter. Cannot proceed.")
        return

    # Step 2: Retrieve documents based on the router's decision
    if user_intent == 'get_course_detail':
        print(f"➡️ Router Decision: METADATA filtering. Filter: {search_filter}")

        # Remove 'user_intent' from filter
        search_filter.pop('user_intent', None)
        search_filter.pop('descriptive_query',None)

        instructors = None
        if "instructors" in search_filter:
            instructors = search_filter['instructors']
            del search_filter['instructors']

        # Defensive: If search_filter is empty or None, use empty dict
        if not search_filter:
            filter_arg = {}
        elif len(search_filter) == 1:
            filter_arg = search_filter
        else:
            # Only use $and if there are at least two filters
            filter_arg = {"$and": [{k: v} for k, v in search_filter.items()]}

        if(search_filter):
            retriever = vector_store.as_retriever(search_kwargs={"filter": filter_arg, "k": 50})
        else:
            retriever = vector_store.as_retriever(search_kwargs={"k": 50})
            
        docs = retriever.get_relevant_documents("")

        if instructors:
            retrieved_docs = [doc for doc in docs if "instructors" in doc.metadata and instructor_fuzzy_match(doc.metadata["instructors"], instructors)]
        else:
            retrieved_docs = docs[:10]

    elif user_intent == 'similar_courses':
        print(f"➡️ Router Decision: METADATA + SEMANTIC search. (SIMILAR COURSES). Filter: {search_filter}")

        del search_filter["user_intent"]
        search_filter.pop('descriptive_query',None)

        if not search_filter:
            retrieved_docs = []
        else:
            retriever = vector_store.as_retriever(search_kwargs = {'filter': search_filter, "k" : 1})
            descriptive_query = retriever.get_relevant_documents("")[0].page_content

            # We can also get the index of the doc and combine the page_content of all the results give it to llm to generate summary of that and then do the semantic search

            retriever = vector_store.as_retriever()
            retrieved_docs = retriever.get_relevant_documents(descriptive_query)

    elif user_intent == 'courses_by_topics':   
        search_filter.pop("user_intent", None)
        descriptive_query = search_filter.pop("descriptive_query", None)

        if descriptive_query is None:
            print("❌ Error: Please provide some more information about the course.")
            return

        if search_filter:
            print(f"➡️ Router Decision: METADATA + SEMANTIC search. (COURSES BY TOPICS). Filter: {search_filter}")
            retrieved_docs = courses_by_topics(search_filter, descriptive_query) 
        else:
            print(f"➡️ Router Decision: Semantic search. Filter: {search_filter}")
            retriever = vector_store.as_retriever()
            retrieved_docs = retriever.get_relevant_documents(descriptive_query) 
    else:
        retrieved_docs = []
        

    # Step 3: Generate the final answer
    context_str = "\n\n---\n\n".join(
        [f"Content: {doc.page_content}\nMetadata: {doc.metadata}" for doc in retrieved_docs]
    )
    chat_history.append(AIMessage(context_str))

    print("💬 Generating final answer...")
    answer = final_rag_chain.invoke({
        "context": context_str,
        "question": query,
        "chat_history" : chat_history  # chat_history is passed here to the prompt
    })

    return answer

# --- 6. LET'S RUN IT! ---
query1 = """ Summarize the  course content of CS 772 in 100 words """
query2 = """ Any course similar to the course i mentioned just now? """
query3 = """ What are the key differnces between them? """

query = query3
chat_history.append(HumanMessage(query))  

# Test a metadata-based query 
answer = ask_course_bot(query)
chat_history.append(AIMessage(answer)) 
print("Course Buddy AI :\n",answer)


🤔 Query: ' What are the key differnces between them? '
Rewritten query: 'What are the key differences between CS 772 and CS 626?'
{'course_code': 'cs772', 'user_intent': 'get_course_detail'} 

⚙️ Search Filter: {'course_code': 'cs772', 'user_intent': 'get_course_detail'}
➡️ Router Decision: METADATA filtering. Filter: {'course_code': 'cs772', 'user_intent': 'get_course_detail'}
💬 Generating final answer...
Course Buddy AI :
 CS 772, "Deep Learning for Natural Language Processing," is explicitly focused on deep learning methodologies and specific neural network architectures like LSTMs, attention, transformers (BERT, GPT), and DNNs for various NLP tasks including text classification, question answering, language generation, and advanced multimodal problems.

In contrast, CS 626, "Speech and Natural Language Processing and the Web," covers a broader range of NLP applications such as sentiment analysis, machine translation, question answering, web applications, analytics, and cross-lingu